# **Features engineering**

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

In [9]:
train = pd.read_csv(r'data/train_final.csv')

## 🚗 Creación de la variable `HasGarage`

Para reflejar explícitamente si una propiedad cuenta con garage o no, se creó la variable binaria `HasGarage`. Esta variable se construyó a partir de características físicas objetivas:

- `GarageArea` > 0
- `GarageCars` > 0

Si alguna de estas condiciones se cumple, `HasGarage = 1`, en caso contrario, `HasGarage = 0`.

Esta variable será útil para modelado, especialmente al combinar con transformaciones polinómicas o análisis de interacciones.

In [10]:
# Crear columna binaria HasGarage basada en si tiene espacio o autos asignados
train['HasGarage'] = (
    (train['GarageArea'] > 0) | 
    (train['GarageCars'] > 0)
).astype(int)


In [11]:
# Filtrar registros sin garage (HasGarage == 0)
sin_garage = train[train['HasGarage'] == 0]

# Verificar que GarageArea, GarageCars y GarageYrBlt sean cero
print("✅ Verificación de registros sin garage:")
print("GarageArea > 0:", (sin_garage['GarageArea'] > 0).sum())
print("GarageCars > 0:", (sin_garage['GarageCars'] > 0).sum())
print("GarageYrBlt > 0:", (sin_garage['GarageYrBlt'] > 0).sum())
print("Total de registros sin garage (HasGarage == 0):", sin_garage.shape[0])


✅ Verificación de registros sin garage:
GarageArea > 0: 0
GarageCars > 0: 0
GarageYrBlt > 0: 0
Total de registros sin garage (HasGarage == 0): 81


In [12]:
cor_gyb = train[train['HasGarage'] == 1][['GarageYrBlt', 'SalePrice']].corr().iloc[0, 1]
print(f"📌 Correlación filtrada GarageYrBlt vs SalePrice (solo con garage): {cor_gyb:.3f}")


📌 Correlación filtrada GarageYrBlt vs SalePrice (solo con garage): 0.548


## 📊 Selección de Variables Numéricas Fuertemente Correlacionadas con `SalePrice`

🎯 **Objetivo**: Seleccionar automáticamente las variables numéricas que presentan una alta correlación con `SalePrice`, para usarlas luego en una expansión polinómica con `PolynomialFeatures`.

📌 **Criterios de selección**:
- Variables **numéricas continuas** (float o int)
- Correlación absoluta con `SalePrice` **mayor a 0.4**
- Se excluyen columnas target, IDs y categóricas codificadas

In [13]:
# Asegurarse de tener solo variables numéricas
numeric_cols = train.select_dtypes(include=['int64', 'float64']).copy()

# Excluir la variable target y cualquier ID
excluded = ['SalePrice', 'Id']
numeric_cols = numeric_cols.drop(columns=[col for col in excluded if col in numeric_cols.columns])

# Calcular correlación con la variable target
correlations = numeric_cols.corrwith(train['SalePrice'])

# Filtrar por correlación fuerte
strong_features = correlations[correlations.abs() > 0.4].sort_values(ascending=False)

# Mostrar resultado
print("📈 Variables numéricas con correlación fuerte (> 0.4) con SalePrice:")
print(strong_features)

# Guardar lista de features para usar en PolynomialFeatures
selected_features = strong_features.index.tolist()


📈 Variables numéricas con correlación fuerte (> 0.4) con SalePrice:
OverallQual          0.823505
GrLivArea            0.733793
GarageCars           0.685087
ExterQual            0.681659
GarageArea           0.668695
KitchenQual          0.668332
TotalBsmtSF          0.639330
BsmtQual             0.614882
GarageFinish         0.606831
1stFlrSF             0.606006
FullBath             0.598591
YearBuilt            0.588787
YearRemodAdd         0.568595
FireplaceQu          0.540808
Foundation_PConc     0.538138
TotRmsAbvGrd         0.536313
Fireplaces           0.484236
HeatingQC            0.477931
BsmtFinType1_GLQ     0.441554
MasVnrArea           0.429103
GarageType_Attchd    0.414782
dtype: float64


## 🔁 Expansión Polinómica de Features Seleccionadas

🎯 **Objetivo**: Generar nuevas variables que representen combinaciones cuadráticas entre las features más correlacionadas con `SalePrice`, usando `PolynomialFeatures`.

📌 **Características del proceso**:
- Se seleccionaron automáticamente variables numéricas con |correlación| > 0.4
- Se aplicó `PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)`
- Esto incluye:
  - Términos cuadráticos (e.g., `GrLivArea²`)
  - Interacciones cruzadas (e.g., `GrLivArea * GarageArea`)
- Se evita incluir un término constante (bias)

In [14]:
# Features seleccionadas
selected_features = strong_features.index.tolist()

# Agregar manualmente GarageYrBlt si no estaba incluida
if 'GarageYrBlt' not in selected_features:
    selected_features.append('GarageYrBlt')

# Subset del dataset con esas columnas
X_poly_base = train[selected_features]

# Crear objeto PolynomialFeatures (cuadrático)
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)

# Aplicar transformación
X_poly = poly.fit_transform(X_poly_base)

# Obtener nombres de las nuevas columnas
poly_feature_names = poly.get_feature_names_out(selected_features)

# Convertir a DataFrame para explorarlo
X_poly_df = pd.DataFrame(X_poly, columns=poly_feature_names)

# Mostrar forma
print(f"✅ Forma del dataset original: {X_poly_base.shape}")
print(f"📈 Forma del dataset expandido: {X_poly_df.shape}")


✅ Forma del dataset original: (1441, 22)
📈 Forma del dataset expandido: (1441, 275)
